The NCBI BLAST servers are a shared resource. We give priority to interactive users. In order to ensure availability of the service to the entire community, we may limit searches for some high volume users. Interactive users of the NCBI webpages through a web browser should not encounter problems. We will move searches of users who submit more than 100 searches in a 24 hour period to a slower queue, or, in extreme cases, will block the requests. To avoid problems, API users should comply with the following guidelines:

* Do not contact the server more often than once every 10 seconds.
* Do not poll for any single RID more often than once a minute.
* Use the URL parameter email and tool, so that the NCBI can contact you if there is a problem.
* Run scripts weekends or between 9 pm and 5 am Eastern time on weekdays if more than 50 searches will be submitted.
* BLAST often runs more efficiently if multiple queries are sent as one search rather than if each query is sent as an individual search. This is especially true for blastn, megablast, and tblastn. If your queries are short (less than a few hundred bases) we suggest you merge them into one search of up to 1,000 bases

The NCBI servers are a shared resource and not intended for projects that involve a large number of BLAST searches. We provide Stand-alone BLAST and the RESTful API at a cloud provider for such projects.

In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [4]:
f = open("conc_protein_seq.fa", "r")
fasta = f.read()

In [5]:
print(fasta)

>PNT58448_Populus_trichocarpa PNT58448 pep chromosome:Pop_tri_v3:1:36133779:36134192:-1 gene:POPTR_001G353600v3 transcript:PNT58448 gene_biotype:protein_coding transcript_biotype:protein_coding description:hypothetical protein
MAFNLKLSLFIAFLACSSLDCYKARARPSASVSNLMARLKLDGDSQNNCWDSLVQLQACS
GEIILFFLNGETQLGRSCCQALRTIGEHCWPNMIDTLGFTAEEGQILEGYCDKAADPTTP
SPPAPSVMPAKVVPKQT
>PNT57551_Populus_trichocarpa PNT57551 pep chromosome:Pop_tri_v3:1:30972858:30973250:1 gene:POPTR_001G306700v3 transcript:PNT57551 gene_biotype:protein_coding transcript_biotype:protein_coding description:hypothetical protein
MDFSFKLLLTFFLTCSTASMPMMAAHPQVSTHTTLATRLRLDNEETTCWGSLLHLQSCIS
NVLLFFLNGETYLRPSCCHAIRIIGHHCWPSMLASLGFTVQEGDILLGYCDATAHSSSPP
PEPIFFPNHT
>PNT57550_Populus_trichocarpa PNT57550 pep chromosome:Pop_tri_v3:1:30971024:30971464:1 gene:POPTR_001G306600v3 transcript:PNT57550 gene_biotype:protein_coding transcript_biotype:protein_coding description:hypothetical protein
MAAFKNLALFLSLTLLISTNISTAARDILINKPGFNSLSARLEDEGSL

In [47]:
with open("conc_protein_seq.fa") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        fasta = str(record.seq)

In [15]:
fasta = """
MFLFLDRTVDEHNPKRVGSCQRDADRDCFESNQIELQEVAHAQFDEWVLVLMKTRCLDNQASDVTPFRLIRENNVEEVGFDISTTHEANRWCTHLLLPQALSTADEEETGYCWDSLMQLQHCWRCWSTMIGALLGFTPQEGDVLQGYSDDDNDSDHKKGDEHALASSPLPLSLKFKPSNVVNP
"""

In [48]:
fasta

'MAPFIKLTLIIFLTCWMASARPLESKPSSLMARLKLEESSGPTCWDSLYELQSCTGEVIQFFYNGESNLEHDCCQAINVIAHNCWPSMLSSLGFTDQETALLQGYCDAEESQAPSPIDN'

In [8]:
def save_content(resp, filename):
    with open(filename, "wb") as file:
        file.write(resp.content)

# Payload

After loading the FASTA sequence into the query, choosing the according database `Whole-genome shotgun contigs (wgs)` and choosing the species `Populus trichocarpa (taxid:3694)`, the recording for the networks bar in inspection mode was started. The `Blast.cgi` originated first and its `Payload` and `Headers` options were explored. We see that the Request URL is https://blast.ncbi.nlm.nih.gov/Blast.cgi.  Without chainging any parameters, it showed me following error:

* Message ID#69 Error: Error occurred while trying to set up a Blast Object from CGI context: CFastaReader: Near line 1, there's a line that doesn't look like plausible data, but it's not marked as defline or comment.

After randomly commenting out some keys, it turned out that `"SUBJECTFILE": "(binary)"` was causing problems - I guess because it referred to the file that was never loaded. 

In [49]:
query_payload = {
"QUERY": fasta,
"db": "protein",
"GENETIC_CODE": 1,
"JOB_TITLE": "",
"ADV_VIEW": "true",
"SUBJECTS": "",
"stype": "nucleotide",
"SUBJECTS_FROM": "",
"SUBJECTS_TO": "",
# "SUBJECTFILE": "(binary)",
"DATABASE": "Whole_Genome_Shotgun_contigs",
"DB_GROUP": "wgsOrg",
"EQ_MENU": "3712",
"NUM_ORG": 1,
"EQ_TEXT": "",
"PHI_PATTERN": "",
"MAX_NUM_SEQ": 100,
"EXPECT": 0.05,
"WORD_SIZE": 6,
"HSP_RANGE_MAX": 0,
"MATRIX_NAME": "BLOSUM62",
"MATCH_SCORES": "1,-2",
"GAPCOSTS": "11 1",
"COMPOSITION_BASED_STATISTICS": 2,
"FILTER": "L",
"REPEATS": 4829,
"TEMPLATE_LENGTH": 0,
"TEMPLATE_TYPE": 0,
"PSSM": "(binary)",
"I_THRESH": "",
"DI_THRESH": "",
"PSI_PSEUDOCOUNT": "", 
"SHOW_OVERVIEW": "true",
"SHOW_LINKOUT": "true",
"GET_SEQUENCE": "true",
"FORMAT_OBJECT": "Alignment",
"FORMAT_TYPE": "HTML",
"ALIGNMENT_VIEW": "Pairwise",
"MASK_CHAR": 2,
"MASK_COLOR": 1,
"DESCRIPTIONS": 100,
"ALIGNMENTS": 100,
"LINE_LENGTH": 60,
"NEW_VIEW": "true",
"NCBI_GI": "false",
"SHOW_CDS_FEATURE": "false",
"NUM_OVERVIEW": 100,
"FORMAT_EQ_TEXT": "",
"FORMAT_ORGANISM": "",
"EXPECT_LOW": "",
"EXPECT_HIGH": "",
"PERC_IDENT_LOW": "",
"PERC_IDENT_HIGH": "",
"QUERY_INDEX": "",
"FORMAT_NUM_ORG": 1,
"CONFIG_DESCR": "2,3,4,5,8,9,10,11,12,13,14",
"CLIENT": "web",
"SERVICE": "plain",
"CMD": "request",
"PAGE": "Translations",
"PROGRAM": "tblastn",
"MEGABLAST": "",
"RUN_PSIBLAST": "", 
"WWW_BLAST_TYPE": "",
"TWO_HITS": "",
"UNGAPPED_ALIGNMENT": "no",
"BLAST_PROGRAMS": "tblastn",
"DB_DISPLAY_NAME": "wgs",
"ORG_DBS": "orgDbsOnly_wgs",
"SHOW_ORGANISMS": "on",
"DBTAXID": "",
"SAVED_PSSM": "",
"SELECTED_PROG_TYPE": "tblastn",
"SAVED_SEARCH": "true",
"BLAST_SPEC": "",
"MIXED_DATABASE": "",
"QUERY_BELIEVE_DEFLINE": "",
"DB_DIR_PREFIX": "",
"CHECKSUM": "",
"USER_DATABASE": "",
"USER_WORD_SIZE": "",
"SER_MATCH_SCORES": "",
"USER_FORMAT_DEFAULTS": "",
"NO_COMMON": "",
"NUM_DIFFS": 1,
"NUM_OPTS_DIFFS": 0,
"UNIQ_DEFAULTS_NAME": "A_SearchDefaults_1nT86e_1Eel_dm10k8GG6pv_GTWlT_1xx6Sk",
"PAGE_TYPE": "BlastSearch",
"USER_DEFAULT_PROG_TYPE": "tblastn",
"USER_DEFAULT_MATRIX": ""}

# Loading the HTML page

Here we download the HTML page directly after query to access the job number, in order to later access the files.

In [50]:
blast_url = "https://blast.ncbi.nlm.nih.gov/Blast.cgi"

query_resp = requests.post(blast_url, data=query_payload)
save_content(query_resp, "query_page.html")

In [51]:
from IPython.display import HTML
HTML(filename='query_page.html')

Request ID,2WEY2GMM01R
Status,Searching
Time since submission,00:00:00


# Accessing the JobID/RID:

When taking a look at the tblant results, there is a row with RID and according number marked blue, when copying the link https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Get&RID=2U55TX9R013, we can see it again consists of the same start `blast_url` and two parameters come to it.

In [56]:
# Opening the previously generated HTML file and reading it into soup
with open('query_page.html') as html_file:
    query_soup = BeautifulSoup(html_file, 'lxml')

# Getting RID

RID = query_soup.find("input", {"name" : "RID"})["value"]

# Making RID payload
rid_payload = {
    "CMD": "Get",
"RID": RID
}

# take accession numbers, make them into align_seq_list and then the page should loud

# Saving the according results
resp = requests.post(blast_url, data=rid_payload)
# print(resp.url)
save_content(resp, "rid.html")

# Accessing the Alignments tab

I first found an old website:

In [ ]:
# Making RID for old website payload
old_rid_payload = {
    "CMD": "Get",
    "RID": RID,
    "ADV_VIEW":"no",
    "CONFIG_DESCR":"2,3,4,5,6,7,8"
}


# Saving the according results
resp = requests.post(blast_url, data=old_rid_payload, stream=False)
print(resp.url)
save_content(resp, "old_rid.html")

In [ ]:
from IPython.display import HTML
HTML(filename='old_rid.html')

After loading the website multiple times and taking a look at Networks section, I noticed that `Alignments` did not load directly and **finally** I noticed that there was actually a separate `t2g.cgi?CMD=Get&RID=2UD5CPGY016&DESCRIPTIONS=0&NUM…K_CHAR=2&MASK_COLOR=1&LINE_LENGTH=60&BOBJSRVC=sra` (accordingly, RIDs changed) and when I clicked on it - only the alignments html showed up. I took a look at the payload and just loaded it in:

In [ ]:
rid_payload = {
    "CMD": "Get",
"RID": RID,
"DESCRIPTIONS": 0,
"NUM_OVERVIEW": 0,
"GET_SEQUENCE": "on",
"DYNAMIC_FORMAT": "on",
"ALIGN_SEQ_LIST": "gb|AARH03000215|,gb|AARH03000658|,gb|AARH03001829|,gb|AARH03002074|,gb|AARH03000170|",
"HSP_SORT": 0,
"SEQ_LIST_START": 1,
"QUERY_INDEX": 0,
'SHOW_LINKOUT': "on",
"ALIGNMENT_VIEW": "Pairwise",
"MASK_CHAR": 2,
"MASK_COLOR": 1,
"LINE_LENGTH": 60,
"BOBJSRVC": "sra"
}

Then I again tried with the trial and error method to see option actually forces the Alignments tab, though quite clear from the start - it's the `ALIGN_SEQ_LIST`, which included the accession numbers. I loaded the accession numbers from the first RID page from the tab `Descriptions` and then combined them in the similar fashion to get the alignments tab:

In [41]:
# Opening the previously generated HTML file and reading it into soup
with open('query_page.html') as html_file:
    query_soup = BeautifulSoup(html_file, 'lxml')

# Getting RID

RID = query_soup.find("input", {"name" : "RID"})["value"]

# Getting accession numbers
rid_payload = {
    "CMD": "Get",
"RID": RID
}

resp = requests.post(blast_url, data=rid_payload)
job_soup = BeautifulSoup(resp.content)
acc_num = []
for i in job_soup.find("table", id="dscTable").tbody.find_all("tr"):
    acc_num.append(i.find("td", class_="c12 l lim").text.strip().split(".")[0])

In [42]:
# take accession numbers, make them into align_seq_list and then the page should load
acc_seq = ""
for i in acc_num:
    acc_seq += "gb|" + i + "|,"

# Making RID payload
alig_payload = {
    "CMD": "Get",
"RID": RID,
"ALIGN_SEQ_LIST": acc_seq[:-1],
}    
    
# Saving the according results
resp = requests.post(blast_url, data=alig_payload)
# print(resp.url)
save_content(resp, "alignments.html")

In [43]:
from IPython.display import HTML
HTML(filename='alignments.html')

Score,Expect,Method,Identities,Positives,Gaps,Frame
165 bits(417),2e-45(),Compositional matrix adjust.,79/80(99%),79/80(98%),0/80(0%),-1
Score,Expect,Method,Identities,Positives,Gaps,Frame
120 bits(301),4e-30(2),Compositional matrix adjust.,87/118(74%),88/118(74%),26/118(22%),-3
Score,Expect,Method,Identities,Positives,Gaps,Frame
33.9 bits(76),4e-30(2),Composition-based stats.,13/15(87%),13/15(86%),0/15(0%),-1
Score,Expect,Method,Identities,Positives,Gaps,Frame
165 bits(417),2e-45(),Compositional matrix adjust.,79/80(99%),79/80(98%),0/80(0%),-2
Score,Expect,Method,Identities,Positives,Gaps,Frame
151 bits(381),1e-40(),Compositional matrix adjust.,103/132(78%),103/132(78%),29/132(21%),-2
Score,Expect,Method,Identities,Positives,Gaps,Frame


# Getting the alignment details

In [ ]:
# Opening the previously generated HTML file and reading it into soup
with open('query_page.html') as html_file:
    query_soup = BeautifulSoup(html_file, 'lxml')

# Getting RID

RID = query_soup.find("input", {"name" : "RID"})["value"]

# Getting accession numbers
rid_payload = {
    "CMD": "Get",
"RID": RID
}

resp = requests.post(blast_url, data=rid_payload)
job_soup = BeautifulSoup(resp.content)
acc_num = []
for i in job_soup.find("table", id="dscTable").tbody.find_all("tr"):
    acc_num.append(i.find("td", class_="c12 l lim").text.strip().split(".")[0])

In [ ]:
# take accession numbers, make them into align_seq_list and then the page should loud
acc_seq = ""
for i in acc_num:
    acc_seq += "gb|" + i + "|,"

# Making RID payload
alig_payload = {
    "CMD": "Get",
"RID": RID,
"ALIGN_SEQ_LIST": acc_seq[:-1],
}    
    
# Saving the according results
resp = requests.post(blast_url, data=alig_payload)
# print(resp.url)
alig_soup = BeautifulSoup(resp.content)

In [ ]:
for i in alig_soup.find_all("div", class_="oneSeqAln"):
    title = i.find("div", class_="dlfRow").text.strip().split("\n")
    subj_name = title[0]
    subtitle = title[1].split(":")
    subj_id = subtitle[1][:-6]
    subj_len = subtitle[2][:-17]
    num_mathes = subtitle[-1]
#     print(subj_name, subj_id, subj_len, num_mathes)
    alig_bar = i.find("div", class_="alnAll")
#     stat_bar = alig_bar.find("div", id=re.compile(r"hd.*"))
#     print(stat_bar.text.strip())
    subj_range = alig_bar.find("span", class_="alnRn").label.text.split()
    subj_range = tuple(map(int, map(subj_range.__getitem__, [-3, -1])))
    score_table = alig_bar.table
    description = [i.text for i in score_table.find_all("tr")[0].find_all("th")]
#     print(score_table.find_all("tr")[0].find_all("th"))
#     print(description)
    scores = [i.text for i in score_table.find_all("tr")[1].find_all("td")]
#     print(scores)
    metrics_dic = dict(zip(description, scores))
#     print(metrics_dic)
    score = metrics_dic["Score"]
    e_value = metrics_dic["Expect"]
    identity = metrics_dic["Identities"]
#     print(score, e_value, identity)
    alig_seq = alig_bar.find("div", id=re.compile(r"ar.*"))
    print(alig_seq.text)
    query_seq = ""
    subj_seq = ""
    spl_alig_seq = alig_seq.text.split()
    for i, entry in enumerate(spl_alig_seq):
        if entry == "Query":
    #         print(i)
    #         print(spl_alig_seq[i+2])
            query_seq += spl_alig_seq[i+2]
        elif entry == "Sbjct":
            subj_seq += spl_alig_seq[i+2]
            
#     print(query_seq)
#     print(subj_seq)
#     break
#     print(alig_bar)

# Getting the taxid number

If we only want to enter the species name, we also have to figure out the taxid number. For that purpose, there is website from NCBI taxonomy https://www.ncbi.nlm.nih.gov/Taxonomy/TaxIdentifier/tax_identifier.cgi. For that purpose, we again take a look at `Networks` tab, see the `tax_identifier.cgi` and load the payload:

In [ ]:
tax_payload = {
    "tax": """human
Cephalosporium acremonium var. radiatum
Arabidopsis thaliana
Homo neanderthalensis
Proboscidea
Bacillus
reptiles
Agathis montana
Mus musacris
Mus muscaris""",
#     "fl": "(binary)",
    "button": "Show on screen"
}

In [ ]:
tax_url = "https://www.ncbi.nlm.nih.gov/Taxonomy/TaxIdentifier/tax_identifier.cgi"
# Saving the according results
resp = requests.post(tax_url, data=tax_payload)
# print(resp.url)
save_content(resp, "tax.html")
from IPython.display import HTML
HTML(filename='tax.html')

In [ ]:
tax_soup = BeautifulSoup(resp.content)

In [ ]:
tax_soup.find("table", cellpadding="0", cellspacing="5", width="600").tr.find_all("td")

In [ ]:
# for i in tax_soup.find("table", cellpadding="0", cellspacing="5", width="600").find_all("tr")[1].find_all("td"):
#     print(i.text)

In [ ]:
for rows in tax_soup.find("table", cellpadding="0", cellspacing="5", width="600").find_all("tr")[1:]:
    code, name, pref_name, taxid = rows.text.split("\n")[:-1]
    print(taxid)

# Finished 

I now finished the hard part and continue formatting in PyCharm in `.py` file.